#### Solution 생성 및 Versioning
솔루선 생성 및 하이퍼 파라미터 조정

In [19]:
import boto3
from botocore.exceptions import ClientError
import json
import time


personalize = boto3.client("personalize")

#### 솔루션 생성

In [15]:
# recipe (알고리즘) 선택
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

In [16]:
# 이미 dataset group이 생성되어 있을 때, 조회
dataset_group_name = "osungmart-dataset-groupmwomk"

list_dataset_group_resp = personalize.list_dataset_groups(
  maxResults=10
)

for dataset_grp in list_dataset_group_resp['datasetGroups']:
  if dataset_grp['name'] == dataset_group_name:
    dataset_group_arn = dataset_grp['datasetGroupArn']
    print("dataset_group_arn: {}".format(dataset_grp['datasetGroupArn']) )

dataset_group_arn: arn:aws:personalize:ap-northeast-2:532805286864:dataset-group/osungmart-dataset-groupmwomk


In [17]:
# create solution
solution_name = "osugmart-user-personalize-solution-HPO"
try: 
  create_solution_resp = personalize.create_solution(
    name = solution_name,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
  )
  solution_arn = create_solution_resp['solutionArn']
  print(json.dumps(create_solution_resp, indent=2))
except ClientError as e:
  print(e)

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:532805286864:solution/osugmart-user-personalize-solution-HTO",
  "ResponseMetadata": {
    "RequestId": "385de6b8-8e6d-498b-89d9-6f623c90b82b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 03 Feb 2023 03:42:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "385de6b8-8e6d-498b-89d9-6f623c90b82b"
    },
    "RetryAttempts": 0
  }
}


In [18]:
# Create Solution Version: training
try:
  create_solution_version_resp = personalize.create_solution_version(
    solutionArn = solution_arn
  )
  solution_version_arn = create_solution_version_resp['solutionVersionArn']
  print(json.dumps(create_solution_version_resp, indent=2))
except ClientError as e:
  print(e)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:532805286864:solution/osugmart-user-personalize-solution-HTO/9a1efd18",
  "ResponseMetadata": {
    "RequestId": "87db8058-8503-4b47-8d4e-16cacd0c3f58",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 03 Feb 2023 03:44:46 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "129",
      "connection": "keep-alive",
      "x-amzn-requestid": "87db8058-8503-4b47-8d4e-16cacd0c3f58"
    },
    "RetryAttempts": 0
  }
}


In [20]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(45)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_P

KeyboardInterrupt: 

#### 성능 평가
AWS User Personalization metrics

In [ ]:
personalize_runtime = boto3.client('personalize-runtime')